# LTI Estimation
There are many situations where it would be beneficial to be able to measure the impulse response of a linear time invariant (LTI) system.  It is not always a good idea to input an impluse as this can cause the system to behave in an undesired way, possibly moving into regions of operation that are not linear.  An alternative is to drive the system with a noise-like input, and use correlation to be able to measure the impulse response.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
from scipy.signal import lfilter
import matplotlib.pyplot as plt

And define a function that will return true if running in a Jupyter Notebook

In [ ]:
def is_jupyter():
    """Return true if running in a Jupyter Notebook"""
    try:
        if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
            return True
        else:
            return False
    except: 
        return False

### User specified parameters
The following parameters can be specified. 

Parameter | Meaning
--------- | -------
<code>sequence_length</code>| The length of the sequence to be used as the input (e.g.10000)
<code>max_correlation_delay</code> | The maximum delay of correlation (should be above the maximum system delay) (e.g. 50)

In [ ]:
sequence_length = 10000
max_correlation_delay = 50

### Define LTI system
Here we define the LTI system that is to be estimated using the correlation technique.  An LTI system is one whose impulse response does not change (the system is time invariant), and that the output can be related to the input through a convolution operation (linear).  So, this function defines the impulse response as an FIR filter, and then uses lfilter from scipy to generate the output of the system for the specified input.  You are encouraged to change the definition of <code>FIR_response</code> to see its effect on the correlation outputs.

In [ ]:
# Unknown_LTI_system defines a linear system to test analysis techniques
# The function takes an input data sequence, x, filters it using an impulse
# response that is known only to the function, and returns the result y.

def Unknown_LTI_system(x):
    """
     Define the linear system.  In this case the system comprises three
     discrete non-zero impulse response values.  This is a typical response
     for a system where multiple echoes of the input are observed at the
     output
     
     INPUT: 
          x: Input data sequence.
          
     RETURN:
          y: The result of filtering input x with an impulse response.
     
    """
    
    FIR_response = np.zeros(26)
    FIR_response[0] = 0.5
    FIR_response[10] = 0.25
    FIR_response[25] = 0.125

    # Run the input through the linear system
    y = lfilter(FIR_response, 1, x)
    return y

For convenience, we define a function to generate the plots

In [ ]:
def plot_fig(X, Y, xlabel, ylabel, xticks, yticks, ylim, title, name):
    """
       Function that create a stem plot and 
       Save the figure with given name when not running in Jupyter notebook
       
       Parameters:
                  X (array-like): The x-positions of the stems. 
                  Y (array-like): The y-positions of the stems. 
                  xlabel (str): The label for the x-axis.
                  ylabel (str): The label for the y-axis.
                  xticks (array-like): The tick locations of the x-axis.
                  yticks (array-like): The tick locations of the y-axis.
                  ylim ([left, right]): Set the ylim to left, right.
                  title (str): The title of figure.
                  name (str): Name used to save figure.
    """
    
    # Create the plot figure
    plt.figure(figsize = (16, 8))
    
    # Enlarge figure label and axis size
    plt.rcParams.update({'font.size': 16})
    
    # stem plot with circle marker 
    (markerLines, stemLines, baseLines) = plt.stem(X, Y, use_line_collection = True) 
    plt.setp(baseLines, color = 'black', linewidth=1) 
    plt.setp(stemLines, linewidth=1) 
    markerLines.set_markersize(4)
    markerLines.set_markerfacecolor('none')
    
    # Tidy up the plot to control axes sizes and labels
    plt.ylim(ylim)
    plt.xlim(X[0],X[-1])
    plt.xticks(xticks)
    plt.yticks(yticks)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    
    # Save figure in python or ipython system
    if not is_jupyter(): plt.savefig(name)

### Input signal
The input is defined as a long noise sequence.  To avoid clipping of very large signals (which is a non-linear effect), it is useful to define the noise signal as uniformly distributed noise between two fixed values.  If, in practice, clipping is observed, then the magnitude of the input should be reduced.

In [ ]:
# First define a random noise input as uniformly distributed white noise
# with a range of -1 to 1
x = 1 - 2 * np.random.rand(sequence_length)

### Plot the first few samples
Just so that we can see what the input looks like, let's plot it.  As you can see, it is random, and is uniformly distributed.

In [ ]:
plot_fig(X = np.arange(0,101), Y = x[0:101],
         xlabel = 'Sample',
         ylabel = 'Amplitude',
         title = 'Input data sequence',
         name = 'LTI_input.pdf',
         ylim = [-1, 1],
         xticks = np.linspace(0, 100, 6),
         yticks = np.linspace(-1, 1, 11))

### Plot the input autocorrelation sequence
One of the properties of autocorrelation for a noise sequence is that it has a non-zero value at zero delay, but at all other delays, the autocorrelation is zero.  Here, as we are dealing with a finite length record of one realisation of the noise signal, the autocorrelation does not fall to zero, but is very close.  If we made the sequence longer, then the peak would get larger, and at all other delays, the autocorrelation would be closer to zero.

In [ ]:
# To avoid calculating all possible delays, which could be very large
# if the input sequence is long, then use pyplot's function xcorr
# which allows a maximum delay to be specified
(lags,input_acf, line, b) = plt.xcorr(x, x, normed = False,
                                      maxlags = max_correlation_delay)
# As we don't actually want the plot, close it afterwards
plt.close()

X = np.arange(-max_correlation_delay, max_correlation_delay+1)

plot_fig(X = X, Y = input_acf,
         xlabel = 'Delay',
         ylabel = 'Correlation',
         ylim = [-500, 3500],
         xticks = np.linspace(-40, 40, 5),
         yticks = np.linspace(-500, 3500, 9),
         title = 'Input autocorrelation sequence',
        name = 'LTI_input_acf.pdf')

### Run the sequence through the unknown system
Now we apply the noise signal to the LTI system.  Note that we do not have direct access to <code>FIR_response</code>, and all we can see is the output of the system, <code>y</code> given the input <code>x</code>.

In [ ]:
y = Unknown_LTI_system(x)

### Plot the first few samples of the output
As we plotted the input signal, then we may as well also plot the first part of the output signal.  As expected, since the input was noise, the output also looks like noise.  

In [ ]:
plot_fig(X = np.arange(0,101), Y = y[0:101],
         xlabel = 'Sample',
         ylabel = 'Amplitude',
         title = 'Output data sequence',
         name = 'LTI_output.pdf',
         ylim = [-0.8, 0.8],
         xticks = np.linspace(0, 100, 11),
         yticks = np.linspace(-0.8, 0.8, 9))

### Compute the crosscorrelation
Now we find the crosscorrelation of the input sequence, which we defined as the noise signal, and the ouptut of the linear system.  We find, when we do this, that the crosscorrelation has the same shape as the system impulse response.  So, it is possible by knowing only the input and output of the system, to be able to determine the impulse response of the system.

In [ ]:
# Again, use pyplot.xcorr to determine the correlation
(lags,io_crosscorrelation, line, b) = plt.xcorr(y, x, normed = False,
                                                maxlags = max_correlation_delay)
plt.close()
X = np.arange(-max_correlation_delay, max_correlation_delay + 1)


plot_fig(X = X, Y = io_crosscorrelation,
         xlabel = 'Delay',
         ylabel = 'Correlation',
         ylim = [-200, 1800],
         xticks = np.linspace(-40, 40, 5),
         yticks = np.linspace(-200, 1800, 11),
         title = 'Crosscorrelation sequence',
         name = 'LTI_ccf.pdf')

### Plot the output autocorrelation sequence
Given the above, we can find the impulse response of the system, but we do need to know both the input and output of the linear system.  The question should be asked, what happens if we only know the output?  In that case, we can compute the autocorrelation.  If the input were a noise like signal (which often it is), then we can still tell something about the system, but obviously it is more complicated than simply the impulse response.  In actual fact, what we obtain is the autocorrelation of the impulse response.

In [ ]:
# calculate the autocorrelation of y
(lags, output_acf, line, b) = plt.xcorr(y, y, normed = False, maxlags = max_correlation_delay)
plt.close()

X = np.arange(-max_correlation_delay, max_correlation_delay + 1)

plot_fig(X = X, Y = output_acf,
         xlabel = 'Delay',
         ylabel = 'Correlation',
         ylim = [-200, 1200],
         xticks = np.linspace(-40, 40, 5),
         yticks = np.linspace(-200, 1200, 8),
         title = 'Output autocorrelation sequence',
         name = 'LTI_output_acf.pdf')

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.